In [1]:
def read_file(path):
    with open(path) as f:
        content = f.readlines()
    return content

In [7]:
wnut_path = "../data/wnut17train.conll"

In [3]:
import pandas as pd
df = pd.read_csv("../data/wnut17train.conll",sep="\t")

In [4]:
labels = []
for i,row in df.iterrows():
    if(row.O != 'O' and (not isinstance(row.O,float))):
        labels.append(row.O)
labels = [x[2:len(x)] for x in labels]
labels = set(labels)

In [5]:
labels

{'corporation', 'creative-work', 'group', 'location', 'person', 'product'}

In [8]:
content = [x.replace("\t", " ") for x in read_file(wnut_path)]
text = [x.split() for x in content]
text[0:5]

[['@paulwalk', 'O'], ['It', 'O'], ["'s", 'O'], ['the', 'O'], ['view', 'O']]

In [9]:
def squash(list_labels):
    res = []
    for i,x in enumerate(list_labels):
        j = 1
        if(i + j< len(list_labels) and x[1] == 'B' and list_labels[i  + j][1] == 'I'):
            while(i + j < len(list_labels) and list_labels[i + j][1] == 'I'):
                j += 1
            res.append((x[0],x[0] + j -1 ,x[2]))
        elif(x[1] == 'B'):
            res.append([x[0],x[2]])
    return res

In [10]:
test = "This is a test".split()

In [11]:
sum([len(x) for x in test[0:2]])

6

In [12]:
def str_i_to_char_i(sent_label):
    res_labels = []
    sent = sent_label[0].split()
    labels = sent_label[1]
    
    for label in labels: 
        if(len(label) == 2):
            start_label_index = label[0]
            end_label_index = label[0]
            label_name = label[1]
        elif(len(label) == 3):
            start_label_index = label[0]
            end_label_index = label[1]
            label_name = label[2]
            


        start_char_index = sum([len(x) for x in sent[0:start_label_index]])    
        end_char_index = start_char_index + sum([len(x) for x in sent[start_label_index:end_label_index + 1]])
        res_labels.append((start_char_index + start_label_index,end_char_index + end_label_index,label_name))
        
            
    return [" ".join(sent), res_labels]

In [13]:
def to_spacy_format(sent_label): 
    sent = sent_label[0]
    labels = sent_label[1]
    res_dict = {'entities': labels}
    return [sent, res_dict]

In [14]:
import nltk
from nltk.tokenize import sent_tokenize 
res = []
sentence = []
sent_labels = []
for x in text:
    if(len(x)!=0):
        sentence.append(x[0])
        sent_labels.append(x[1])
    else:
        sent_labels = [(i,x[0], x[2:len(x)]) for i,x in enumerate(sent_labels) if x != 'O' ]
        sent_labels = squash(sent_labels)
        res.append([" ".join(sentence),sent_labels])
        sent_labels = []
        sentence = []
        
res = [str_i_to_char_i(x) for x in res]
res = [to_spacy_format(x) for x in res]
TRAIN_DATA = res


In [17]:
import random
import spacy
model = None
n_iter=10

if model is not None:
    nlp1 = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp1 = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")

#create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp1.pipe_names:
    print("Add ner pipe")
    ner = nlp1.create_pipe('ner')
    nlp1.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp1.get_pipe('ner')


Created blank 'en' model
Add ner pipe


In [273]:
x ="~~hey guys, i got this switch up for grabs, the website is my switch patched shows that it is hackable, but i believe the latest update has been applied so may not be hackable anymore? i dont really know.\n\nI had given this to my son last xmass but then he asked for a laptop so in part payment he gave me the switch. under his possession he always had a screen protector in it. It does however have like 3 or 4 small scratches that are not visible while playing. The body overall has normal signs of use like light scratches and what not but nothing crazy. The games included are Smash Ultimate, Mariokart 8 Deluxe and Minecraft. It does have a new pair of joy cons because the old left one the bumper is bad, maybe an easy fix, but didn't bother and just bought another set. The old are included too. The battery seems to be holding a charge just fine. I just dont use it, rather play games on my pc as well.\n\nI do have the original box, and what i think is most of the accessories check pics for all the is included. \n\npictures: https://imgur.com/a/Rr3jRoJ\n\nLooking for 280  locally (HOUSTON- GALVESTON area) \n\nshipped looking for about 300 to cover shipping.  Rather sell locally so sort of firm on the shipped price.~~\n\nSOLD"

In [274]:
TRAIN_DATA = [
    (x, {
        'entities': [(23, 29, 'PRODUCT'), (62, 68, 'PRODUCT'), (309, 315, 'PRODUCT'), (411, 426, 'CONDITION'), (509, 524, 'CONDITION'), (1031, 1067, 'URL'),
                    (1086, 1109, 'LOCATION'), (1072, 1084, 'LOCALPRICE')]
    })]

In [18]:
# add labels, Trains data based on annotations 
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [25]:
from tqdm import tqdm
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp1.pipe_names if pipe != 'ner']

with nlp1.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp1.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp1.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.5,  # dropout 
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

  0%|          | 4/3394 [00:00<01:54, 29.53it/s]

{'ner': 4750.463819609098}


100%|██████████| 3394/3394 [01:54<00:00, 29.68it/s]

{'ner': 4508.24149024284}


In [20]:
# test the trained model
for text, _ in TRAIN_DATA:
    doc = nlp1(x)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

TypeError: Argument 'string' has incorrect type (expected str, got list)